In [ ]:
from langchain_community.llms import Ollama
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.agents import Tool, initialize_agent, AgentType
from langchain.chains import RetrievalQA
import os

In [3]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("./untitled.txt")
docs = loader.load()


In [4]:
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(docs)

In [6]:
embedding_model = OllamaEmbeddings(model="llama3.2") 
db = FAISS.from_documents(chunks, embedding_model)
retriever = db.as_retriever()

/var/folders/26/5b37wfvn5p3c2jwdpnt8g_280000gn/T/ipykernel_45082/1138023187.py:1: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding_model = OllamaEmbeddings(model="llama3.2")


In [7]:
rag_chain = RetrievalQA.from_chain_type(
    llm=Ollama(model="llama3.2"),
    retriever=retriever,
    return_source_documents=True
)

/var/folders/26/5b37wfvn5p3c2jwdpnt8g_280000gn/T/ipykernel_45082/1764437846.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm=Ollama(model="llama3.2"),


In [8]:
rag_tool = Tool(
    name="HRPolicyQA",
    func=rag_chain.run,
    description="Useful for answering questions about company HR policies and leave rules."
)

In [9]:
llm = Ollama(model="llama3.2")

agent = initialize_agent(
    tools=[rag_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

/var/folders/26/5b37wfvn5p3c2jwdpnt8g_280000gn/T/ipykernel_45082/3572685311.py:3: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [ ]:
while True:
    query = input("\nAsk me anything about HR Policy (or type 'exit'): ")
    if query.lower() in ['exit', 'quit']:
        print("👋 Exiting HR Policy Assistant. Have a great day!")
        break

    try:
        response = agent.invoke(query)
        print("\n🧠 Answer:\n" + response)
    except Exception as e:
        print(f"\n⚠️ Error: {str(e)}")


Ask me anything about HR Policy (or type 'exit'):  How many sick leave?




> Entering new AgentExecutor chain...
Action: HRPolicyQA
Action Input: How many sick leave?
⚠️ Error: `run` not supported when there is not exactly one output key. Got ['result', 'source_documents'].
